In [ ]:
import os
import contextlib

import pandas as pd
import seaborn as sns
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
def find_sheet_name(sheet_names):
    cpi_sheet_names = list(filter(lambda x: "cpi" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    if len(sheet_names) == 43:
        return "30"

    if len(sheet_names) == 11:
        return "Sheet2"

    raise RuntimeError(sheet_names)

In [ ]:
en_to_vn = {
    "CONSUMER PRICE INDEXES": "CHỈ SỐ GIÁ TIÊU DÙNG",
    "Food and catering services": "Hàng ăn và dịch vụ ăn uống",
    "Grain food": "Lương thực",
    "Foodstuff": "Thực phẩm",
    "Outdoor eating and drinking": "Ăn uống ngoài gia đình",
    "Drinks and tobacco": "Đồ uống và thuốc lá",
    "Textile, footgear and hats": "May mặc, mũ nón và giày dép ",
    "Housing and construction materials": "Nhà ở và vật liệu xây dựng",
    "Family appliances and tools": "Thiết bị và đồ dùng gia đình",
    "Medicaments and health service": "Thuốc và dịch vụ y tế",
    "Medical service": "Dịch vụ y tế",
    "Transport": "Giao thông",
    "Postal and communicational service": "Bưu chính viễn thông",
    "Education": "Giáo dục",
    "Educational service": "Dịch vụ giáo dục",
    "Culture, entertainment and tourism": "Văn hoá, giải trí và du lịch",
    "Others": "Hàng hóa và dịch vụ khác",
    "GOLD PRICE INDEXES": "CHỈ SỐ GIÁ VÀNG",
    "US DOLLAR PRICE INDEXES": "CHỈ SỐ GIÁ ĐÔ LA MỸ",
    "CORE INFLATION": "LẠM PHÁT CƠ BẢN",
}

In [ ]:
def get_cpi(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Hàng ăn và dịch vụ ăn uống")
                | (df[df.columns[1]] == "Food and catering services")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Lương thực")
                | (df[df.columns[1]] == "Grain food")
                | (df[df.columns[1]] == " Lương thực")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    if len(df.columns) == 7:
        df = df.drop(columns=[df.columns[1], df.columns[3], df.columns[5]])
    elif len(df.columns) == 6 and int(date.split("-")[1]) == 12:
        df = df.drop(columns=[df.columns[1], df.columns[4]])
    elif len(df.columns) == 6:
        df = df.drop(columns=[df.columns[1], df.columns[3]])
    elif len(df.columns) == 4 and int(date.split("-")[1]) == 1:
        df = df.drop(columns=[df.columns[1]])
        df["A"] = df[df.columns[1]]
    elif len(df.columns) == 5 and int(date.split("-")[1]) == 12:
        df = df.drop(columns=[df.columns[1]])
    else:
        raise RuntimeError(df.columns)

    df.columns = ["Hàng hóa", "YOY", "MOM", "YTD-YOY"]

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "CHỈ SỐ GIÁ TIÊU DÙNG")
                    | ((df[df.columns[0]] == "CONSUMER PRICE INDEXES"))
                ].index[0]
            )
        )
    )

    if len(df[df.columns[0]][df[df.columns[0]] == "CONSUMER PRICE INDEXES"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    # df = df[pd.to_numeric(df["MOM"], errors="coerce").notnull()]
    df = df[pd.to_numeric(df["YOY"], errors="coerce").notnull()]
    df = df[pd.to_numeric(df["YTD-YOY"], errors="coerce").notnull()]

    df["MOM"] = df["MOM"].astype(float) - 100
    df["YOY"] = df["YOY"].astype(float) - 100
    df["YTD-YOY"] = df["YTD-YOY"].astype(float) - 100

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    return df


xl = pd.ExcelFile("../datas/2019-12-27-03-Bieu-2019.xlsx")
get_cpi(xl, find_sheet_name(xl.sheet_names), "2019-12-27")

In [ ]:
import os
import contextlib

dfs_cpi_origin = []
for file_name in sorted(
    os.listdir("../datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("../datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2014:
            continue

        xl = pd.ExcelFile(file_path)

        df_cpi = get_cpi(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_cpi_origin.append(df_cpi)
    except Exception as e:
        print(file_name)
        raise e

In [ ]:
dfs_cpi = pd.concat(dfs_cpi_origin, ignore_index=True, axis=0)
dfs_cpi["Date"] = pd.to_datetime(dfs_cpi["Date"], dayfirst=False)
dfs_cpi.head(n=20)

In [ ]:
convert_hh = {
    "CHỈ SỐ GIÁ TIÊU DÙNG": "CHỈ SỐ GIÁ TIÊU DÙNG",
    "Hàng ăn và dịch vụ ăn uống": "Hàng ăn và dịch vụ ăn uống",
    "Lương thực": "Lương thực",
    "Thực phẩm": "Thực phẩm",
    "Ăn uống ngoài gia đình": "Ăn uống ngoài gia đình",
    "Đồ uống và thuốc lá": "Đồ uống và thuốc lá",
    "May mặc, mũ nón và giày dép": "May mặc, mũ nón và giày dép",
    "Nhà ở và vật liệu xây dựng": "Nhà ở và vật liệu xây dựng",
    "Thiết bị và đồ dùng gia đình": "Thiết bị và đồ dùng gia đình",
    "Thuốc và dịch vụ y tế": "Thuốc và dịch vụ y tế",
    "Dịch vụ y tế": "Dịch vụ y tế",
    "Giao thông": "Giao thông",
    "Bưu chính viễn thông": "Bưu chính viễn thông",
    "Giáo dục": "Giáo dục",
    "Dịch vụ giáo dục": "Dịch vụ giáo dục",
    "Văn hoá, giải trí và du lịch": "Văn hoá, giải trí và du lịch",
    "Hàng hóa và dịch vụ khác": "Hàng hóa và dịch vụ khác",
    "CHỈ SỐ GIÁ VÀNG": "CHỈ SỐ GIÁ VÀNG",
    "CHỈ SỐ GIÁ ĐÔ LA MỸ": "CHỈ SỐ GIÁ ĐÔ LA MỸ",
    "LẠM PHÁT CƠ BẢN": "LẠM PHÁT CƠ BẢN",
    "May mặc, giày dép và mũ nón": "May mặc, mũ nón và giày dép",
    "Đồ dùng và dịch vụ khác": "Hàng hóa và dịch vụ khác",
    "May mặc, mũ nón và giày dép ": "May mặc, mũ nón và giày dép",
    "LẠM PHÁT CƠ BẢN(*)": "LẠM PHÁT CƠ BẢN",
}

dfs_cpi["Hàng hóa"] = dfs_cpi["Hàng hóa"].map(convert_hh)
dfs_cpi

In [ ]:
dfs_cpi.loc[dfs_cpi["Hàng hóa"].str.contains("LẠM PHÁT CƠ BẢN"), "YOY"] = (
    dfs_cpi.loc[dfs_cpi["Hàng hóa"].str.contains("LẠM PHÁT CƠ BẢN"), "YOY"] + 100
)
dfs_cpi.loc[dfs_cpi["Hàng hóa"].str.contains("LẠM PHÁT CƠ BẢN"), "MOM"] = (
    dfs_cpi.loc[dfs_cpi["Hàng hóa"].str.contains("LẠM PHÁT CƠ BẢN"), "MOM"] + 100
)
dfs_cpi.loc[dfs_cpi["Hàng hóa"].str.contains("LẠM PHÁT CƠ BẢN"), "YTD-YOY"] = (
    dfs_cpi.loc[dfs_cpi["Hàng hóa"].str.contains("LẠM PHÁT CƠ BẢN"), "YTD-YOY"] + 100
)

In [ ]:
dfs_cpi.to_csv("../csv/cpi.csv")

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_cpi[
    (
        (dfs_cpi["Hàng hóa"] == "CHỈ SỐ GIÁ TIÊU DÙNG")
        | (dfs_cpi["Hàng hóa"] == "LẠM PHÁT CƠ BẢN")
    )
    & (dfs_cpi["Date"] > "2015-01-01")
]

fig = px.area(df_show, x="Date", y="YOY", color="Hàng hóa")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_cpi[
    (
        (dfs_cpi["Hàng hóa"] == "Lương thực")
        | (dfs_cpi["Hàng hóa"] == "Nhà ở và vật liệu xây dựng")
        | (dfs_cpi["Hàng hóa"] == "Giao thông")
        | (dfs_cpi["Hàng hóa"] == "Giáo dục")
        | (dfs_cpi["Hàng hóa"] == "Thuốc và dịch vụ y tế")
    )
    & (dfs_cpi["Date"] > "2021-01-01")
]

fig = px.area(df_show, x="Date", y="YOY", color="Hàng hóa")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()